In [2]:
In [49]:
import warnings
warnings.filterwarnings('ignore')
In [50]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
In [51]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
Read the CSV and Perform Basic Data Cleaning
In [52]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]
In [53]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()
Out[53]:
loan_amnt	int_rate	installment	home_ownership	annual_inc	verification_status	issue_d	loan_status	pymnt_plan	dti	...	pct_tl_nvr_dlq	percent_bc_gt_75	pub_rec_bankruptcies	tax_liens	tot_hi_cred_lim	total_bal_ex_mort	total_bc_limit	total_il_high_credit_limit	hardship_flag	debt_settlement_flag
0	10500.0	0.1719	375.35	RENT	66000.0	Source Verified	Mar-2019	low_risk	n	27.24	...	85.7	100.0	0.0	0.0	65687.0	38199.0	2000.0	61987.0	N	N
1	25000.0	0.2000	929.09	MORTGAGE	105000.0	Verified	Mar-2019	low_risk	n	20.23	...	91.2	50.0	1.0	0.0	271427.0	60641.0	41200.0	49197.0	N	N
2	20000.0	0.2000	529.88	MORTGAGE	56000.0	Verified	Mar-2019	low_risk	n	24.26	...	66.7	50.0	0.0	0.0	60644.0	45684.0	7500.0	43144.0	N	N
3	10000.0	0.1640	353.55	RENT	92000.0	Verified	Mar-2019	low_risk	n	31.44	...	100.0	50.0	1.0	0.0	99506.0	68784.0	19700.0	76506.0	N	N
4	22000.0	0.1474	520.39	MORTGAGE	52000.0	Not Verified	Mar-2019	low_risk	n	18.76	...	100.0	0.0	0.0	0.0	219750.0	25919.0	27600.0	20000.0	N	N
5 rows × 86 columns
In [54]:
# 1- Encoding text columns:

df = pd.get_dummies(df, columns=['home_ownership', 'verification_status', 'issue_d', 'pymnt_plan', 'initial_list_status', 'next_pymnt_d', 'application_type', 'hardship_flag', 'debt_settlement_flag'])
df.head(2)
Out[54]:
loan_amnt	int_rate	installment	annual_inc	loan_status	dti	delinq_2yrs	inq_last_6mths	open_acc	pub_rec	...	issue_d_Mar-2019	pymnt_plan_n	initial_list_status_f	initial_list_status_w	next_pymnt_d_Apr-2019	next_pymnt_d_May-2019	application_type_Individual	application_type_Joint App	hardship_flag_N	debt_settlement_flag_N
0	10500.0	0.1719	375.35	66000.0	low_risk	27.24	0.0	0.0	8.0	0.0	...	1	1	0	1	0	1	1	0	1	1
1	25000.0	0.2000	929.09	105000.0	low_risk	20.23	0.0	0.0	17.0	1.0	...	1	1	0	1	0	1	1	0	1	1
2 rows × 96 columns
Split the Data into Training and Testing
In [55]:
# Create our features
x_col = [col for col in df.columns if col not in ['loan_status']]
X = df[x_col]
# Create our target
y = df.loan_status
In [56]:
X.describe()
Out[56]:
loan_amnt	int_rate	installment	annual_inc	dti	delinq_2yrs	inq_last_6mths	open_acc	pub_rec	revol_bal	...	issue_d_Mar-2019	pymnt_plan_n	initial_list_status_f	initial_list_status_w	next_pymnt_d_Apr-2019	next_pymnt_d_May-2019	application_type_Individual	application_type_Joint App	hardship_flag_N	debt_settlement_flag_N
count	68817.000000	68817.000000	68817.000000	6.881700e+04	68817.000000	68817.000000	68817.000000	68817.000000	68817.000000	68817.000000	...	68817.000000	68817.0	68817.000000	68817.000000	68817.000000	68817.000000	68817.000000	68817.000000	68817.0	68817.0
mean	16677.594562	0.127718	480.652863	8.821371e+04	21.778153	0.217766	0.497697	12.587340	0.126030	17604.142828	...	0.177238	1.0	0.123879	0.876121	0.383161	0.616839	0.860340	0.139660	1.0	1.0
std	10277.348590	0.048130	288.062432	1.155800e+05	20.199244	0.718367	0.758122	6.022869	0.336797	21835.880400	...	0.381873	0.0	0.329446	0.329446	0.486161	0.486161	0.346637	0.346637	0.0	0.0
min	1000.000000	0.060000	30.890000	4.000000e+01	0.000000	0.000000	0.000000	2.000000	0.000000	0.000000	...	0.000000	1.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1.0	1.0
25%	9000.000000	0.088100	265.730000	5.000000e+04	13.890000	0.000000	0.000000	8.000000	0.000000	6293.000000	...	0.000000	1.0	0.000000	1.000000	0.000000	0.000000	1.000000	0.000000	1.0	1.0
50%	15000.000000	0.118000	404.560000	7.300000e+04	19.760000	0.000000	0.000000	11.000000	0.000000	12068.000000	...	0.000000	1.0	0.000000	1.000000	0.000000	1.000000	1.000000	0.000000	1.0	1.0
75%	24000.000000	0.155700	648.100000	1.040000e+05	26.660000	0.000000	1.000000	16.000000	0.000000	21735.000000	...	0.000000	1.0	0.000000	1.000000	1.000000	1.000000	1.000000	0.000000	1.0	1.0
max	40000.000000	0.308400	1676.230000	8.797500e+06	999.000000	18.000000	5.000000	72.000000	4.000000	587191.000000	...	1.000000	1.0	1.000000	1.000000	1.000000	1.000000	1.000000	1.000000	1.0	1.0
8 rows × 95 columns
In [57]:
# Check the balance of our target values
y.value_counts()
Out[57]:
low_risk     68470
high_risk      347
Name: loan_status, dtype: int64
In [58]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Data Pre-Processing
Scale the training and testing data using the StandardScaler from sklearn. Remember that when scaling the data, you only scale the features data (X_train and X_testing).
In [59]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
In [60]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

X_scaler = scaler.fit(X_train)
In [61]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
Ensemble Learners
In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:
Train the model using the training data.
Calculate the balanced accuracy score from sklearn.metrics.
Print the confusion matrix from sklearn.metrics.
Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score
Note: Use a random state of 1 for each algorithm to ensure consistency between tests
Balanced Random Forest Classifier
In [62]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
model = model.fit(X_train_scaled, y_train)
In [63]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)
Out[63]:
0.7887512850910909
In [64]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)
Out[64]:
array([[   71,    30],
       [ 2146, 14958]], dtype=int64)
In [65]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.63     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.63     17205

In [66]:
# List the features sorted in descending order by feature importance
importances = model.feature_importances_
importances_sorted = sorted(zip(model.feature_importances_, X.columns), reverse=True)
importances_sorted
Out[66]:
[(0.07876809003486353, 'total_rec_prncp'),
 (0.05883806887524815, 'total_pymnt'),
 (0.05625613759225244, 'total_pymnt_inv'),
 (0.05355513093134745, 'total_rec_int'),
 (0.0500331813446525, 'last_pymnt_amnt'),
 (0.02966959508700077, 'int_rate'),
 (0.021129125328012987, 'issue_d_Jan-2019'),
 (0.01980242888931366, 'installment'),
 (0.01747062730041245, 'dti'),
 (0.016858293184471483, 'out_prncp_inv'),
 (0.01641297102011915, 'total_bal_ex_mort'),
 (0.015220714904737209, 'mths_since_recent_inq'),
 (0.015115240704562424, 'issue_d_Mar-2019'),
 (0.014926655663448373, 'revol_bal'),
 (0.014899352873994727, 'max_bal_bc'),
 (0.014881069023035237, 'out_prncp'),
 (0.014859446582326507, 'mo_sin_old_rev_tl_op'),
 (0.014832564501144122, 'annual_inc'),
 (0.014613819728800227, 'bc_util'),
 (0.014487685026878092, 'tot_hi_cred_lim'),
 (0.013921085423763812, 'mo_sin_old_il_acct'),
 (0.013534131593418711, 'mths_since_rcnt_il'),
 (0.013364759441576994, 'total_rev_hi_lim'),
 (0.01332289882475225, 'avg_cur_bal'),
 (0.013265926832893358, 'bc_open_to_buy'),
 (0.01311545089813887, 'il_util'),
 (0.01304530062898567, 'total_bc_limit'),
 (0.0130446065288952, 'all_util'),
 (0.013030046723135838, 'pct_tl_nvr_dlq'),
 (0.012855901280381887, 'tot_cur_bal'),
 (0.01279908506759016, 'total_il_high_credit_limit'),
 (0.012773576514405109, 'inq_last_12m'),
 (0.011968994260747247, 'total_bal_il'),
 (0.010982948025240226, 'mo_sin_rcnt_tl'),
 (0.010579906006851516, 'loan_amnt'),
 (0.010575363106694519, 'mths_since_recent_bc'),
 (0.010320067009550682, 'num_rev_accts'),
 (0.010209212170253059, 'total_acc'),
 (0.009753839399393215, 'open_acc_6m'),
 (0.009662050208879065, 'num_il_tl'),
 (0.009632472481996241, 'num_actv_rev_tl'),
 (0.009393346012674945, 'mo_sin_rcnt_rev_tl_op'),
 (0.00872448189550355, 'num_bc_tl'),
 (0.008628938824946404, 'num_sats'),
 (0.008330966254402506, 'num_op_rev_tl'),
 (0.007570544824579072, 'num_bc_sats'),
 (0.007548811505974241, 'num_tl_op_past_12m'),
 (0.007489717491934961, 'open_act_il'),
 (0.007382231721841728, 'open_acc'),
 (0.007272665006598051, 'num_rev_tl_bal_gt_0'),
 (0.006998827313196186, 'total_rec_late_fee'),
 (0.006866662924995743, 'num_actv_bc_tl'),
 (0.006714495620628373, 'acc_open_past_24mths'),
 (0.006561432872333855, 'open_il_24m'),
 (0.006240598451492287, 'inq_fi'),
 (0.006216409633238659, 'inq_last_6mths'),
 (0.0061708920490257954, 'mort_acc'),
 (0.006083218608279307, 'total_cu_tl'),
 (0.005640206440873574, 'issue_d_Feb-2019'),
 (0.005634546230136711, 'open_rv_24m'),
 (0.005131046989565006, 'percent_bc_gt_75'),
 (0.005106000423451099, 'next_pymnt_d_Apr-2019'),
 (0.005036652777545191, 'next_pymnt_d_May-2019'),
 (0.004860024796675963, 'open_il_12m'),
 (0.004198582835532627, 'tot_coll_amt'),
 (0.004018916067963884, 'open_rv_12m'),
 (0.0037571920083085985, 'num_accts_ever_120_pd'),
 (0.003082852259926947, 'delinq_2yrs'),
 (0.0029133221443170495, 'home_ownership_RENT'),
 (0.002824523629114469, 'home_ownership_OWN'),
 (0.002204946377565813, 'verification_status_Not Verified'),
 (0.0020912385738361574, 'application_type_Joint App'),
 (0.002015258269512615, 'pub_rec'),
 (0.0019325773153555006, 'verification_status_Verified'),
 (0.001901604006185586, 'application_type_Individual'),
 (0.0015046400907840708, 'verification_status_Source Verified'),
 (0.0014589723334940362, 'home_ownership_MORTGAGE'),
 (0.0013727925120781853, 'pub_rec_bankruptcies'),
 (0.0011520703643731528, 'initial_list_status_w'),
 (0.0011005704165634263, 'initial_list_status_f'),
 (0.0009956935704327383, 'num_tl_90g_dpd_24m'),
 (0.0007150315534652695, 'collections_12_mths_ex_med'),
 (0.0004955956183545533, 'home_ownership_ANY'),
 (0.0002730803587770788, 'chargeoff_within_12_mths'),
 (0.0, 'tax_liens'),
 (0.0, 'recoveries'),
 (0.0, 'pymnt_plan_n'),
 (0.0, 'policy_code'),
 (0.0, 'num_tl_30dpd'),
 (0.0, 'num_tl_120dpd_2m'),
 (0.0, 'hardship_flag_N'),
 (0.0, 'delinq_amnt'),
 (0.0, 'debt_settlement_flag_N'),
 (0.0, 'collection_recovery_fee'),
 (0.0, 'acc_now_delinq')]
Easy Ensemble Classifier
In [67]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

model2= EasyEnsembleClassifier(n_estimators=100, random_state=1)
model2 = model2.fit(X_train_scaled, y_train)
In [72]:
# Calculated the balanced accuracy score
y_pred = model2.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)
Out[72]:
0.931601605553446
In [73]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)
Out[73]:
array([[   93,     8],
       [  985, 16119]], dtype=int64)
In [74]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

In [ ]:


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 171)